# Trabalho Big Data - Dimensionalidade 👌

## Bibliotecas

In [76]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Importando os dados

In [59]:
data = pd.read_csv('Cancer_Data.csv')
data = data.drop(['id', 'Unnamed: 32'], axis=1)
data.shape

(569, 31)

## Tratando outliers

In [82]:
data_t = data.copy()
colunms = data_t.columns.drop(['diagnosis'])

for col in colunms:
    Q3 = data_t[col].quantile(0.75)
    Q1 = data_t[col].quantile(0.25)

    IQR = Q3 - Q1

    data_t = data_t[(data_t[col] >= Q1 - 1.5*IQR) & (data_t[col] <= Q3 + 1.5*IQR)]

data_t.replace({'M': 1, 'B': 0}, inplace=True)

data_t = pd.DataFrame(data_t, columns=data.columns)
data_t.shape

(277, 31)

## Modelo de classificação usando o dataset inteiro, sem redução de dimensionalidade

In [86]:
X = data_t.drop(['diagnosis'], axis=1)
y = data_t['diagnosis']

X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

model_1 = LogisticRegression()
model_1.fit(X_train, y_train)

y_pred = model_1.predict(X_test)

acc = accuracy_score(y_test, y_pred)

f"O modelo sem a redução de dimensionalidade teve uma acurácia de {acc:.2%}"

'O modelo sem a redução de dimensionalidade teve uma acurácia de 95.71%'

## Redução da dimensionalidade e modelo de classificação

Aqui utilizamos três métodos de dimensionalidade, comparando cada um deles e utilizando o mesmo modelo de classificação.

1. SelectKBest

In [88]:
from sklearn.feature_selection import SelectKBest, f_classif

data_KB = data_t.copy()

X = data_t.drop(['diagnosis'], axis=1)
y = data_t['diagnosis']

selector = SelectKBest(f_classif, k=15)
X_new = selector.fit_transform(X, y)

selected_columns = X.columns[selector.get_support()]

selected_features = pd.DataFrame(X_new, columns=selected_columns)

In [89]:
X = selected_features
X.shape, y.shape

((277, 15), (277,))

In [90]:
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

In [91]:
model = LogisticRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f"O modelo, com a redução da dimensionalidade dos dados, teve uma acurácia de {acc:.2%}"

'O modelo, com a redução da dimensionalidade dos dados, teve uma acurácia de 98.57%'

2. 